In [71]:
import pandas as pd
import numpy as np
import datetime
import requests
import re

In [72]:
pip install bs4

In [74]:
from bs4 import BeautifulSoup

## Define crawling function

In [75]:
def crawl(stn, mm, x, y, prefix='https://www.weather.go.kr/weather/climate/past_cal.jsp?'):
    website = requests.get(prefix+'stn='+str(stn)+'&yy=2019&mm='+str(mm)+'&'+str(x)+'&'+str(y)+'&'+'obs=1')
    whole = website.content
    one = BeautifulSoup(whole, 'html.parser')
    part = one.find_all('td')

    content = []

    for child in part:
        stats = child.get_text('|')
        content.append(stats.split('|'))

    content_real = []
    for a in content:
        if len(a) > 1:
            content_real.append(a)

    aver = []
    high = []
    low = []
    rain = []

    for child in content_real:
        a = child[0].replace('℃',':')
        a = a.split(':')
        aver.append(a[1])
    
        b = child[1].replace('℃',':')
        b = b.split(':')
        high.append(b[1])
        
        c = child[2].replace('℃',':')
        c = c.split(':')
        low.append(c[1])
        
        if (child[4]=='일강수량: - '):
            d = 0
            rain.append(d)
        else:
            d = child[4].replace('mm',':')
            d = d.split(':')
            rain.append(d[1])

    final = pd.DataFrame({'aver':aver, 'high':high, 'low':low, 'rain':rain})
    
    month = {}
    month[str(mm)+'_'+str(stn)] = final
    
    return month

## Crawl the data

In [76]:
seoul_1 = crawl(stn=108, mm=1, x=15, y=4)['1_108']
seoul_2 = crawl(stn=108, mm=2, x=15, y=4)['2_108']
seoul_3 = crawl(stn=108, mm=3, x=15, y=4)['3_108']
seoul_4 = crawl(stn=108, mm=4, x=15, y=4)['4_108']
seoul_5 = crawl(stn=108, mm=5, x=15, y=4)['5_108']
seoul_6 = crawl(stn=108, mm=6, x=15, y=4)['6_108']
seoul_7 = crawl(stn=108, mm=7, x=15, y=4)['7_108']
seoul_8 = crawl(stn=108, mm=8, x=15, y=4)['8_108']
seoul_9 = crawl(stn=108, mm=9, x=15, y=4)['9_108']
seoul_10 = crawl(stn=108, mm=10, x=15, y=4)['10_108']
seoul_11 = crawl(stn=108, mm=11, x=15, y=4)['11_108']
seoul_12 = crawl(stn=108, mm=12, x=15, y=4)['12_108']

## Create date data (2019.01.01 ~ 2019.12.31)

In [77]:
jan = []
start = datetime.date(2019,1,1)
jan.append(start)
for i in range(30):
    start = start + datetime.timedelta(days=1)
    jan.append(start)

    
feb = []
start = datetime.date(2019,2,1)
feb.append(start)
for i in range(27):
    start = start + datetime.timedelta(days=1)
    feb.append(start)

    
mar = []
start = datetime.date(2019,3,1)
mar.append(start)
for i in range(30):
    start = start + datetime.timedelta(days=1)
    mar.append(start)

    
apr = []
start = datetime.date(2019,4,1)
apr.append(start)
for i in range(29):
    start = start + datetime.timedelta(days=1)
    apr.append(start)

    
may = []
start = datetime.date(2019,5,1)
may.append(start)
for i in range(30):
    start = start + datetime.timedelta(days=1)
    may.append(start)

    
jun = []
start = datetime.date(2019,6,1)
jun.append(start)
for i in range(29):
    start = start + datetime.timedelta(days=1)
    jun.append(start)

    
jul = []
start = datetime.date(2019,7,1)
jul.append(start)
for i in range(30):
    start = start + datetime.timedelta(days=1)
    jul.append(start)

    
aug = []
start = datetime.date(2019,8,1)
aug.append(start)
for i in range(30):
    start = start + datetime.timedelta(days=1)
    aug.append(start)

    
sep = []
start = datetime.date(2019,9,1)
sep.append(start)
for i in range(29):
    start = start + datetime.timedelta(days=1)
    sep.append(start)

    
oct = []
start = datetime.date(2019,10,1)
oct.append(start)
for i in range(30):
    start = start + datetime.timedelta(days=1)
    oct.append(start)

    
nov = []
start = datetime.date(2019,11,1)
nov.append(start)
for i in range(29):
    start = start + datetime.timedelta(days=1)
    nov.append(start)

    
dec = []
start = datetime.date(2019,12,1)
dec.append(start)
for i in range(30):
    start = start + datetime.timedelta(days=1)
    dec.append(start)

## Create columns ('temp_diff_s' & 'rain')

In [78]:
seoul = pd.concat([seoul_1,seoul_2,seoul_3,seoul_4,seoul_5,seoul_6,seoul_7,seoul_8,seoul_9,seoul_10,seoul_11,seoul_12], axis=0, ignore_index=True)
date = jan+feb+mar+apr+may+jun+jul+aug+sep+oct+nov+dec
seoul.insert(0,'ymd',date)

seoul[['aver','high','low','rain']] = seoul[['aver','high','low','rain']].astype(float)

seoul['temp_diff_s'] = seoul.high - seoul.low
seoul.loc[seoul.rain < 5, 'rain'] = 0
seoul.loc[seoul.rain >= 5, 'rain'] = 1
seoul.drop(['aver','high','low'], axis=1, inplace=True)

## Export

In [80]:
seoul.to_csv('C:/Users/secre/Desktop/bigcon/champ/weather/weather_final.csv', index=False)